In [51]:
import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS글로벌,001250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
2,HSD엔진,082740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
3,LG이노텍,011070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시
4,OCI,010060,기초 화학물질 제조업,"타르제품,카본블랙,무수프탈산,농약원제,석탄화학제품,정밀화학제품,플라스틱창호재 제조,판매",1985-07-09,12월,"백우석, 이우현, 김택중(3인, 각자 대표이사)",http://www.oci.co.kr,서울특별시


In [83]:
import requests
from bs4 import BeautifulSoup

def get_url(item_name, code_df):
    code = code_df.query("회사명=='{}'".format(item_name))['종목코드'].to_string(index=False).strip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    
    print("요청 URL = {}".format(url))
    return url

item_name='DSR'
url = get_url(item_name, code_df)
res = requests.get(url)

soap = BeautifulSoup(res.text, 'lxml')
pg_last = int(soap.find("table", class_="Nnavi").find("td", class_="pgRR").a.get('href').rsplit('&')[1].split('=')[1])

df = pd.DataFrame()

for page in range(1, pg_last):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)

# df.dropna()를 이용해 결측값 있는 행 제거
df = df.dropna()

# 데이터의 타입을 int형으로 바꿔줌
df[['종가', '전일비', '시가', '고가', '저가', '거래량']] \
    = df[['종가', '전일비', '시가', '고가', '저가', '거래량']].astype(int)

# 컬럼명 'date'의 타입을 date로 바꿔줌
df['날짜'] = pd.to_datetime(df['날짜'])

# 일자(date)를 기준으로 오름차순 정렬
df = df.sort_values(by=['날짜'], ascending=True)

# 상위 5개 데이터 확인
df.head()

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=155660


,날짜,종가,전일비,시가,고가,저가,거래량
2443,2013-05-24,5460,50,5630,5750,5340,1429636
2442,2013-05-27,5540,80,5470,5580,5420,637980
2441,2013-05-28,5620,80,5540,5620,5350,888954
2440,2013-05-29,5310,310,5710,5780,5310,970528
2439,2013-05-30,5100,210,5350,5390,5100,585149


In [127]:
import elasticsearch
import json

es_client = elasticsearch.Elasticsearch("localhost:9200", http_auth = ("elastic", "changeme"))


row = df.values.tolist()[0]

company_info = code_df.query("회사명=='{}'".format(item_name)).to_dict('records')[0]




company_info = code_df.query("회사명=='{}'".format(item_name)).to_dict('records')[0]

for price_info in df.to_dict('records'):
    doc = {}; index_date = price_info['날짜'].strftime("%Y-%m")
    index_name = f'stock_price-{index_date}'
    doc.update(company_info)
    doc.update(price_info)
    es_client.index(index=index_name, body=doc)
    print(index_name)
    print(doc)
    break











stock_price-2013-05
{'회사명': 'DSR', '종목코드': '155660', '업종': '1차 비철금속 제조업', '주요제품': '합섬섬유로프', '상장일': '2013-05-15', '결산월': '12월', '대표자명': '홍석빈', '홈페이지': 'http://www.dsr.com', '지역': '부산광역시', '날짜': Timestamp('2013-05-24 00:00:00'), '종가': 5460, '전일비': 50, '시가': 5630, '고가': 5750, '저가': 5340, '거래량': 1429636}


In [ ]:
import plotly.offline as offline 
import plotly.graph_objs as go 

# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True)

trace = go.Scatter(
                x=df.날짜,
                y=df.종가,
                name=item_name)

data = [trace]

# data = [celltrion]
layout = dict(
            title=f'{item_name}의 종가(close) Time Series',
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                             label='1m',
                             step='month',
                             stepmode='backward'),
                        dict(count=3,
                             label='3m',
                             step='month',
                             stepmode='backward'),
                        dict(count=6,
                             label='6m',
                             step='month',
                             stepmode='backward'),
                        dict(step='all')
                    ])
                ),
                rangeslider=dict(),
                type='date'
            )
        )

fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)